In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

In [ ]:
#install PRAW package (Reddit API wrapper) 
#!pip install praw

In [ ]:
#install PSAW package (API wrapper containing workaround for previous API)
#!pip install psaw

In [ ]:
import praw 
from psaw import PushshiftAPI

In [ ]:
#key in based on api application on reddit
reddit = praw.Reddit(client_id='', 
                     client_secret='', 
                     user_agent=''
                    )

api = PushshiftAPI(reddit)

In [ ]:
celeb_list = ['Ben Platt', 'Caitlyn Jenner', 'Ellen Degeneres','Elliot Page','James Charles','Janelle Monáe', 'Jeffree Star', 
         'Joey Graceffa', 'Lady Gaga', 'Lil Nas X', 'Lili Reinhart', 'Miley Cyrus', 'Ronen Rubinstein', 'Sam Smith', 
         'Tom Daley', 'Ariana Grande', 'Billie Eilish', 'Selena Gomez', 'Barack Obama', 'Jaden Smith', 'Kanye West', 'Lizzo',
         'Michael B. Jordan', 'Michelle Obama', 'Nicki Minaj', 'Oprah', 'Rihanna', 'Serena Williams', 'Simone Biles', 
         'Stephan James', 'Terry Crews', 'Tyler Perry', 'Willow Smith', 'Will Smith', 'Dwayne Johnson', 'Daniel Dae Kim',
         'Sandra Oh', 'Alex Shibutani', 'Maia Shibutani', 'Awkwafina', 'Ken Jeong', 'Lana Condor', 'Olivia Rodrigo',
         'Bill Gates', 'Kim Kardashian West', 'Paris Hilton', 'Taylor Swift', 'Tom Hanks', 'Justin Bieber', 'Joe Biden']

In [ ]:
all_date = {'jan20':[(2020,1,15),(2020,2,6)],
        'feb20':[(2020,2,7),(2020,3,29)],
        'mar20':[(2020,3,30),(2020,4,22)],
        'apr20':[(2020,4,23),(2020,5,4)],
        'may20':[(2020,5,5),(2020,6,27)],
        'jun20':[(2020,6,28),(2020,7,1)],
        'jul20':[(2020,7,2),(2020,8,21)],
        'aug20':[(2020,8,22),(2020,9,8)],
        'sep20':[(2020,9,9),(2020,10,11)],
        'oct20':[(2020,10,12),(2020,11,10)],
        'nov20':[(2020,11,11),(2020,12,16)],
        'dec20':[(2020,12,17),(2020,12,31)]}

In [ ]:
def getUnix(year,month,day):
    return int(dt.datetime(year, month, day).timestamp())

In [ ]:
all_date_unix = {}

for key in all_date:
    all_date_unix[key] = [getUnix(all_date[key][0][0], all_date[key][0][1], all_date[key][0][2]),
                     getUnix(all_date[key][1][0], all_date[key][1][1], all_date[key][1][2])]

In [ ]:
all_date_unix

In [ ]:
def getSID(search_term, start, end):
    max_reponse = 50
    submission_id = []
    search = api.search_submissions(q=search_term, after=start, before=end)
    for s in search:
        submission_id.append(s.id)
        if len(submission_id) >= max_reponse:
            break
    return submission_id

In [ ]:
#function gets all the comment from the particular post/submission
def getComments(sid):
    submission = reddit.submission(id=sid)
    comments_list = []
    submission.comments.replace_more(limit=None) #return secondary level replies
    for comment in submission.comments.list():
        timestamp = dt.datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        comments_list.append([timestamp,sid,comment.body])
    return comments_list

In [ ]:
def monthly_comment(id_list):
    all_comments = []
    
    for id in id_list:
        if len(getComments(id)) == 0:
            pass
        else:
            all_comments+=(getComments(id))
            
    return all_comments
    

In [ ]:
mypath = "."

for celeb in celeb_list:
    entire_comment = [] 
    
    for month in all_date_unix:
        all_id = getSID(celeb, all_date_unix[month][0], all_date_unix[month][1])
        entire_comment += monthly_comment(all_id)
        print()
        
    output_csv = f"{mypath}/reddit_{celeb}.csv"
    #print(entire_comment)
    pd.DataFrame(entire_comment, columns = ['Timestamp','SubmissionID', 'Comment']).to_csv(output_csv)
    
    break